this is a markdown comment. the ipynb format is useful for getting individual cells to run

In [107]:
x = "Here we go!"
print(x)

Here we go!


In [108]:
import sys
print(sys.version)


3.13.5 (v3.13.5:6cb20a219a8, Jun 11 2025, 12:23:45) [Clang 16.0.0 (clang-1600.0.26.6)]


In [109]:
# Cell 1 - Imports and Setup
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import process
import pandas as pd
import time
from tqdm import tqdm

headers = {'User-Agent': 'aditya.goel.tx@gmail.com'}


In [110]:
# Cell 2 - Load Clean Fund List CSV
df_funds = pd.read_csv("Clean_Fund_List.csv")
df_funds

,fund_group,fund_id,name,ticker,fund_stkid,advt,fund_type
0,fund,04B8BC-E,American Beacon Balanced Fund,ABCCX US,135909,0,MF_O
1,fund,04B8BD-E,American Beacon Large Cap Value Fund,AADEX US,25667,0,MF_O
2,fund,04B8BK-E,RBC Enterprise Fund,TETAX US,84097,0,MF_O
3,fund,04B8BQ-E,Cohen & Steers Real Estate Securities Fund,CREFX US,210588,0,MF_O
4,fund,04B8BY-E,Voya Growth & Income Portfolio,ISVGX US,119362,0,MF_O
...,...,...,...,...,...,...,...
4837,fund,10L5FM-E,Naa World Equity Income Fund,SFGCX US,67247,0,MF_O
4838,fund,10MBT4-E,Kensington Dynamic Growth,KAGCX US,264944,0,MF_O
4839,fund,10ZJSS-E,Vontobel US Equity Fund,VNUYX US,224821,0,MF_O
4840,fund,112M3V-E,New Age Alpha SMid Cap Value Fund,SEVSX US,67504,0,MF_O


In [111]:

def get_cik_from_mutual_fund(fund_name):
    base_url = "https://www.sec.gov/cgi-bin/series"
    params = {
        'company': fund_name,
        'sc': 'companyseries',
        'ticker': '',
        'CIK': '',
        'type': 'N-PX'
    }

    headers = {'User-Agent': 'aditya.goel.tx@gmail.com'}
    response = requests.get(base_url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Get all CIK-style links from the results table
    cik_links = soup.select('td a[href*="CIK="]')
    
    base_cik = None
    best_series_cik = None
    best_score = -1

    for link in cik_links:
        cik_text = link.text.strip()
        label_text = link.find_parent('td').find_next_sibling('td').text.strip().lower()

        # Detect base CIK
        if cik_text.isdigit():
            base_cik = cik_text

        # Detect series CIKs starting with 'S'
        elif cik_text.upper().startswith('S'):
            # Use overlap logic to choose best one
            score = sum(1 for word in fund_name.lower().split() if word in label_text)
            if score > best_score:
                best_score = score
                best_series_cik = cik_text

    return base_cik, best_series_cik


In [112]:
# Try the first fund in your list
test_name = df_funds.iloc[10]['name'] 
print("Testing fund:", test_name)

# Try retrieving CIK
cik = get_cik_from_mutual_fund(test_name)
print("CIK found:", cik)


Testing fund: Columbia Acorn Fund
CIK found: ('0000002110', 'S000033622')


In [113]:

results = []
save_every = 25
for idx, fund in tqdm(enumerate(df_funds['name']), total=len(df_funds)):
    try:
        base_cik, series_cik = get_cik_from_mutual_fund(fund)
        results.append({
            'Fund_Name': fund,
            'CIK': base_cik,
            'Series_CIK': series_cik
        })
        print(f"[{idx}] ✅ {fund} → Base: {base_cik} | Series: {series_cik}")
    except Exception as e:
        print(f"[{idx}] ❌ Error for {fund}: {e}")
        results.append({
            'Fund_Name': fund,
            'CIK': None,
            'Series_CIK': None
        })
    time.sleep(0.2)  # polite delay to not hammer SEC

    if idx % save_every == 0 and idx != 0:
        partial_df = pd.DataFrame(results)
        partial_df.to_csv('Fund_CIK_Mappings_partial.csv', index=False)


  0%|          | 0/4842 [00:00<?, ?it/s]

[0] ✅ American Beacon Balanced Fund → Base: 0000809593 | Series: S000000718


  0%|          | 1/4842 [00:00<24:42,  3.26it/s]

[1] ✅ American Beacon Large Cap Value Fund → Base: 0000809593 | Series: S000001091


  0%|          | 2/4842 [00:00<25:20,  3.18it/s]

...

100%|█████████▉| 4840/4842 [34:24<00:00,  2.35it/s]

[4840] ✅ New Age Alpha SMid Cap Value Fund → Base: 0002013853 | Series: S000087631


100%|█████████▉| 4841/4842 [34:24<00:00,  2.54it/s]

[4841] ✅ SEGALL BRYANT & HAMILL TRUST - Barrett Opportunity Fund → Base: None | Series: None


100%|██████████| 4842/4842 [34:25<00:00,  2.34it/s]


In [114]:
# Save to file
df_result = pd.DataFrame(results)
df_result.to_csv("Fund_CIK_Series_Mapped.csv", index=False)
print("✅ Finished and saved to Fund_CIK_Series_Mapped.csv")

✅ Finished and saved to Fund_CIK_Series_Mapped.csv


Completed: scraping the CIKs and Series CIKs for MF_O funds from the SEC website
Next: getting the actual filings for each fund from the SEC API

In [ ]:
# Here, we are getting the list of NPORT-P filings available for a given company

#### Will add code to get ciks from the csv file. 
fund_cik = "0000809593"  # Example: American Beacon Balanced Fund
# format the CIK to be 10 digits long (SEC requires zero-padding)
formatted_cik = fund_cik.zfill(10)

# Build the SEC EDGAR API URL to get recent submissions (filings) for the fund
submissions_url = f"https://data.sec.gov/submissions/CIK{formatted_cik}.json"
response = requests.get(submissions_url, headers=headers)  # Make the HTTP request to fetch the JSON data from SEC
data = response.json() # parse the JSON response # SEC API only allows the 1000 most recent submissions. 

filings = pd.DataFrame(data["filings"]["recent"]) # convert the 'recent filings' section into a pandas DataFrame
# filter the filings to only keep "NPORT-P" forms (monthly public portfolio holdings)
nport_df = filings[filings["form"] == "NPORT-P"].copy()

# Define a helper function to construct the direct URL to the filing document
def build_doc_url(cik, acc_no, doc_name):
    acc_no_nodashes = acc_no.replace("-", "")  # SEC URLs require accession numbers without dashes
    return f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{acc_no_nodashes}/{doc_name}"

# Apply the URL builder function to EACH row in the filtered DataFrame. 
# lambda is a shortcut for making a tiny function
nport_df["url"] = nport_df.apply(
    lambda row: build_doc_url(formatted_cik, row["accessionNumber"], row["primaryDocument"]),
    axis=1
)

# Select and organize final useful columns into a clean format
nport_df = nport_df[["form", "accessionNumber", "filingDate", "url"]]

nport_df


,form,accessionNumber,filingDate,url
1,NPORT-P,0001752724-25-161816,2025-06-27,https://www.sec.gov/Archives/edgar/data/809593...
2,NPORT-P,0001752724-25-161815,2025-06-27,https://www.sec.gov/Archives/edgar/data/809593...
3,NPORT-P,0001752724-25-161814,2025-06-27,https://www.sec.gov/Archives/edgar/data/809593...
4,NPORT-P,0001752724-25-161812,2025-06-27,https://www.sec.gov/Archives/edgar/data/809593...
5,NPORT-P,0001752724-25-161811,2025-06-27,https://www.sec.gov/Archives/edgar/data/809593...
...,...,...,...,...
987,NPORT-P,0001752724-21-023922,2021-02-16,https://www.sec.gov/Archives/edgar/data/809593...
996,NPORT-P,0001752724-21-010516,2021-01-26,https://www.sec.gov/Archives/edgar/data/809593...
997,NPORT-P,0001752724-21-010515,2021-01-26,https://www.sec.gov/Archives/edgar/data/809593...
998,NPORT-P,0001752724-21-010440,2021-01-26,https://www.sec.gov/Archives/edgar/data/809593...


Some important notes about the code above:

The SEC only allow 1000 filings to be read through the API
The filings we get for a fund may include various filings for different series of that fund (same filing date). For now, the summary will be driven at a larger mutual fund level, ignoring the differences in the various series/versions of the fund. Later, we can consider adding a further, finer version of insights that include the differences in series. (In many cases, fund managers don't even create separate strategies for the different series. They are simply different fee structures.)